# Collision Avoidance - Live Demo

In this notebook we'll use the model we trained to detect whether the robot is ``free`` or ``blocked`` to enable a collision avoidance behavior on the robot.  

## Load the trained model

We'll assumed that you've already downloaded the ``best_model.pth`` to your workstation as instructed in the training notebook.  Now, you should upload this model into this notebook's
directory by using the Jupyter Lab upload tool.  Once that's finished there should be a file named ``best_model.pth`` in this notebook's directory.  

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

In [ ]:
!sudo systemctl restart nvargus-daemon

[sudo] password for jetson: 

In [1]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224)
# camera = USBCamera(width=224, height=224, capture_device=1)
camera.running=True

image = widgets.Image(format='jpeg', width=224, height=224)
# blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
robot_action_Text = widgets.Text(description='Action: ')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, robot_action_Text]))

In [2]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 4)

Next, load the trained weights from the ``best_model.pth`` file that you uploaded

In [3]:
model.load_state_dict(torch.load('best_model.pth'))

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [4]:
device = torch.device('cuda')
model = model.to(device)

### Create the preprocessing function

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesnt *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

In [5]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Great! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability that the robot is blocked.

We'll also create our robot instance which we'll need to drive the motors.

In [6]:
from jetracer.nvidia_racecar import NvidiaRacecar
robot = NvidiaRacecar()
# car.manual = 1


Next, we'll create a function that will get called whenever the camera's value changes.  This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. While the neural network output indicates we're blocked, we'll turn left, otherwise we go forward.

In [7]:
import torch.nn.functional as F
import time
import ipywidgets

def goright(throttle):
    robot.steering_gain = -0.8
    robot.steering = 1.0
    robot.steering_offset = 0.2
    robot.throttle_gain = 1.0
    robot.throttle = throttle

def gostraight(throttle):
    robot.steering_gain = -0.8
    robot.steering = 0.0
    robot.steering_offset = 0.2
    robot.throttle_gain = 1.0
    robot.throttle = throttle

def goleft(throttle):
    robot.steering_gain = -0.8
    robot.steering = -1.0
    robot.steering_offset = 0.2
    robot.throttle_gain = 1.0
    robot.throttle = throttle
    
def stop(throttle):
    robot.steering_gain = -0.8
    robot.steering = 0.0
    robot.steering_offset = 0.2
    robot.throttle_gain = 0.0
    robot.throttle = throttle

def update(change):
    global robot_action_Text, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    robot_action = float(y.flatten()[0])
#    print(robot_action)
    robot_action_Text.value = str(robot_action)
    
    if robot_action < 0.25 and robot_action >=0:
        goright(-0.3)
    elif  robot_action < 0.5 and robot_action >=0.25:
        gostraight(-0.3)
    elif  robot_action < 0.75 and robot_action >=0.5:
        goleft(-0.3)
    else:
        stop(-0.0)
        
update({'new': camera.value})  # we call the function once to intialize

0.01766357012093067


Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing. 

We accomplish that with the ``observe`` function.

> WARNING: This code will move the robot!! Please make sure your robot has clearance.  The collision avoidance should work, but the neural
> network is only as good as the data it's trained on!

In [8]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

0.02709319069981575
0.0025628255680203438
0.005971068050712347
0.012934477999806404
0.003624602919444442
0.002025782596319914
0.005277446936815977
0.0026519191451370716
0.007069324608892202
0.00735956896096468
0.002701487625017762
0.004039283841848373
0.03811374306678772
0.006840861402451992
0.024494096636772156
0.007987181656062603
0.003183256136253476
0.0022208583541214466
0.015591409057378769
0.004801488947123289
0.0012716528726741672
0.007911551743745804
0.002819678047671914
0.011079555377364159
0.003759360872209072
0.00166232546325773
0.005682199262082577
0.02780255116522312
0.0040223253890872
0.021030759438872337
0.013561230152845383
0.005148594733327627
0.005655515473335981
0.0011374682653695345
0.024341795593500137
0.012030520476400852
0.0015702532837167382
0.0006440000724978745
0.020373912528157234
0.03539232537150383
0.003720496315509081
0.004645336885005236
0.018789688125252724
0.005007187370210886
0.003577978815883398
0.007847938686609268
0.004536082968115807
0.005645947530

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame.  Perhaps start by placing your robot on the ground and seeing what it does when it reaches an obstacle.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [11]:
import time

# camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.throttle_gain = 0.0
robot.throttle = 0.0

Perhaps you want the robot to run without streaming video to the browser.  You can unlink the camera as below.

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)

To continue streaming call the following.

In [ ]:
camera_link.link()  # stream to browser (wont run camera)

### Conclusion

That's it for this live demo!  Hopefully you had some fun and your robot avoided collisions intelligently! 

If your robot wasn't avoiding collisions very well, try to spot where it fails.  The beauty is that we can collect more data for these failure scenarios
and the robot should get even better :)